In [8]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import linear_model
import gzip
from collections import defaultdict

In [9]:
dataset = []
with open("RAW_interactions.csv", encoding="utf8") as f:
    next(f)  
    for l in f:
        fields = l.strip().split(",")  # Remove trailing newline and split by commas
        dataset.append(fields)

In [10]:
dataset[0]
#user_id, recipe_id, date, rating, review

['38094',
 '40893',
 '2003-02-17',
 '4',
 'Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks.']

In [14]:
for row in dataset:
    recipe_id = row[1]  # Assuming recipe_id is the second field
    rating = row[3]     # Assuming rating is the fourth field
    #print("Recipe id: " + recipe_id)
    #print("Rating: " + rating)

IndexError: list index out of range